<a href="https://colab.research.google.com/github/harttu/nlpHarkka/blob/master/introNLPloppuprojekti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Intro to NLP coursework

##Getting data  from the server
* Lemmatized version performs slightly better


In [0]:
useLemmatization = False

In [0]:
file = ""
if useLemmatization:
  !wget https://raw.githubusercontent.com/harttu/nlpHarkka/master/outputLemmaUniq.tsv
  file = "outputLemmaUniq.tsv"
else:
  !wget https://raw.githubusercontent.com/harttu/nlpHarkka/master/outputNormiUniq.tsv
  file = "outputNormiUniq.tsv"

--2020-03-17 17:57:28--  https://raw.githubusercontent.com/harttu/nlpHarkka/master/outputNormiUniq.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 285600 (279K) [text/plain]
Saving to: ‘outputNormiUniq.tsv.1’

outputNormiUniq.tsv 100%[===================>] 278.91K  --.-KB/s    in 0.03s   

2020-03-17 17:57:28 (9.36 MB/s) - ‘outputNormiUniq.tsv.1’ saved [285600/285600]



## Building datamodel 
 * Including neutral label, makes model perform very poorly
 

In [0]:
import json
import random
import csv
import numpy as np

data = []
with open(file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='\t')
    #line_count = 0
    for row in csv_reader:
      # get rid of the neutral class
      if row[0] != "neutral":
        data.append({"class":row[0],"aspect":row[1],"before":row[2],"subject":row[3],"after":row[4], "text":row[2]+" "+row[3]+" "+row[4]})

#data = dict(filter(lambda x: True if x.class != "neutral" else False, data.items()))
random.shuffle(data) #play it safe! (why?)
print("class label:", data[0]["class"])
print("text:",data[0]["text"])

class label: negative
text: Ei mäcissä vikaa ole , mutta monivarsi on parempi .  Mäcci  on kustannustehokas kompromissi . Monivarsi voidaan kustomoida paremmin eri automalleihin , mäcciä ei .


In [0]:
# We need to gather the texts and labels into separate lists
texts=[one_example["text"] for one_example in data]
afters = [one_example["after"] for one_example in data]
subjects = [one_example["subject"] for one_example in data]
befores = [one_example["before"] for one_example in data]

labels=[one_example["class"] for one_example in data]

print("This many texts",len(texts))
print("This many labels",len(labels))
print()
for label,text in list(zip(labels,texts))[:20]:
    print(label,text[:50]+"...")



This many texts 672
This many labels 672

negative Ei mäcissä vikaa ole , mutta monivarsi on parempi ...
positive  Suzukisi  oli nelari mutta juuri tollaista jotaki...
positive Ainakin  Citymarketeissa  oon nähnyt , mutta ekoka...
positive Itselleni tulee ensiviikolla postissa  MPC Studio ...
negative Ei se ihan niinkään ole . Java 7 on nyt se juttu j...
positive Kerrotko jonkun ominaisuuden vim:ssä mitä ei saa e...
positive >pakko ostaa Samsungia hieman hymyilin ääneen tuos...
positive Soijasuikaleita on itse tullut vedettyä kanan korv...
negative No bemarilla taisi rajoittua suurimmat ongelmat m5...
positive Lisätään vielä että liikenteen prioriteetti on var...
positive Kyllä tämä on enemmänkin Grimea kuin Memphisiä . A...
negative Muistellaanko yhdessä kuinka reilua on verrata kel...
positive Onhan niitä , mutta kunnolliset ohjelmat esim  ear...
positive > Tosin ei varmaan mukavuuksilta samaa tasoa kuin ...
negative  740i : ssä  ei ole turboja joten niiden vaihtamin...
positive Siis

In [0]:
#vectorizer=TfidfVectorizer(max_features=100000,binary=True,ngram_range=(1,1))
#feature_matrix=vectorizer.fit_transform(texts)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer



vectorizer=CountVectorizer(max_features=100000,binary=True,ngram_range=(1,1))
feature_matrix=vectorizer.fit_transform(texts)
print("shape=",feature_matrix.shape)
print("what did we get? ->", feature_matrix.__class__)

shape= (672, 8113)
what did we get? -> <class 'scipy.sparse.csr.csr_matrix'>


In [0]:
print(vectorizer.get_feature_names()[:1000])

['04', '07', '10', '100', '1000', '100e', '100km', '100tkm', '10e', '10min', '11', '1100', '112cdi', '112db', '12', '120', '1200e', '121', '123', '12c', '13', '130', '134a', '139e', '14', '140', '142mm', '144mm', '14e', '15', '150', '1500e', '15tkm', '16', '16gb', '16x', '17', '175000', '18', '189e', '19', '1919', '1978', '1982', '1991', '1992', '1993', '19v', '1a', '1ma', '20', '200', '2000', '2000kg', '2002', '2004', '2006', '2009', '2012', '2013', '2014', '2015', '2016', '2020', '205', '20v', '21', '211', '23', '24', '242', '248', '25', '250', '250hp', '256', '26', '265', '26kansi', '27', '280', '280x', '284', '2e', '2k', '2kk', '2paloa', '30', '300', '300hp', '300tkm', '301', '308', '30tkm', '320', '325', '325i', '328', '33', '330', '3331', '350', '3632qm', '38e', '3d', '3kpl', '3literisellä', '3x', '40', '4000s', '400e', '400tkm', '407', '42', '420p', '43', '448', '45', '45e', '47', '4get', '4ghz', '4k', '50', '500', '500e', '500gb', '50cm', '50g', '520', '55', '550w', '560', '5e'

# Data split



In [0]:
from sklearn.model_selection import train_test_split

train_texts, dev_texts, train_labels, dev_labels=train_test_split(texts,labels,test_size=0.2,stratify=labels)
vectorizer=CountVectorizer(max_features=100000,binary=True,ngram_range=(1,1))
feature_matrix_train=vectorizer.fit_transform(train_texts)
feature_matrix_dev=vectorizer.transform(dev_texts)

In [0]:
print(feature_matrix_train.shape)
print(feature_matrix_dev.shape)

(537, 6644)
(135, 6644)


# Classifier train

* Let us try the venerable, if a bit outdated SVM
* Linear SVM for simplicity

In [0]:
import sklearn.svm
classifier=sklearn.svm.LinearSVC(C=50,verbose=1)
classifier.fit(feature_matrix_train, train_labels)

[LibLinear]

LinearSVC(C=50, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=1)

In [0]:
classifier.fit(feature_matrix_train, train_labels)
print("DEV",classifier.score(feature_matrix_dev, dev_labels))
print("TRAIN",classifier.score(feature_matrix_train, train_labels))

[LibLinear]DEV 0.6
TRAIN 1.0


In [0]:
import sklearn.metrics
predictions_dev=classifier.predict(feature_matrix_dev)
#print(predictions_dev)
print(sklearn.metrics.confusion_matrix(dev_labels,predictions_dev))
print(sklearn.metrics.accuracy_score(dev_labels,predictions_dev))

[[18 31]
 [23 63]]
0.6


## Toy data for benchmarking

In [0]:
TOYtexts = texts
random.shuffle(TOYtexts)
TOYlabels = labels
#random.shuffle(TOYlabels)
# classifier
TOYclassifier=sklearn.svm.LinearSVC(C=0.5,verbose=1)
# data split
TOYtrain_texts, TOYdev_texts, TOYtrain_labels, TOYdev_labels=train_test_split(TOYtexts,TOYlabels,test_size=0.2,stratify=TOYlabels)
# vectorizer
TOYvectorizer=CountVectorizer(max_features=100000,binary=True,ngram_range=(1,1))
# training
TOYfeature_matrix_train=TOYvectorizer.fit_transform(TOYtrain_texts)
# apply model
TOYfeature_matrix_dev=TOYvectorizer.transform(TOYdev_texts)
# let the model try to do its best
TOYclassifier.fit(TOYfeature_matrix_train, TOYtrain_labels)
# examine scores
print("TOYDEV",TOYclassifier.score(TOYfeature_matrix_dev, TOYdev_labels))
print("TOYTRAIN",TOYclassifier.score(TOYfeature_matrix_train, TOYtrain_labels))
# for confusion matrix
TOYpredictions_dev=TOYclassifier.predict(TOYfeature_matrix_dev)
print(sklearn.metrics.confusion_matrix(TOYdev_labels,TOYpredictions_dev))

[LibLinear]TOYDEV 0.562962962962963
TOYTRAIN 1.0
[[12 37]
 [22 64]]


*Performance is about 50 % as it should be with randomized data